In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [3]:
from haystack.utils import launch_es
launch_es()


In [32]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "152.67.165.13")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)


Traceback (most recent call last):
  File "c:\Users\rohit\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "c:\Users\rohit\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\util\connection.py", line 96, in create_connection
    raise err
  File "c:\Users\rohit\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\util\connection.py", line 86, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\rohit\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 255, in perform_request
    response 

ConnectionError: Initial connection to Elasticsearch failed. Make sure you run an Elasticsearch instance at `[{'host': '152.67.165.13', 'port': 9200}]` and that it has finished the initial ramp up (can take > 30s).

In [5]:
# from haystack.document_stores import InMemoryDocumentStore

# document_store = InMemoryDocumentStore(use_bm25=True)


In [6]:
doc_dir = r"zui"

In [7]:
from haystack.utils import convert_files_to_docs
from haystack.nodes import PreProcessor


all_docs = convert_files_to_docs(dir_path=doc_dir)
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

INFO:haystack.utils.preprocessing:Converting zui\glcoa_test_cases.txt
INFO:haystack.utils.preprocessing:Converting zui\New-ConvertRite_Installation_Guide_Ver(1.2) (1).docx
INFO:haystack.utils.preprocessing:Converting zui\RiteSync Guide_new.docx
INFO:haystack.utils.preprocessing:Converting zui\Revised ConvertRite User Manual - New_V3.pdf
Preprocessing: 100%|██████████| 4/4 [00:00<00:00, 123.67docs/s]

n_files_input: 4
n_docs_output: 49


In [8]:
from haystack.document_stores import FAISSDocumentStore


document_store = FAISSDocumentStore(sql_url="sqlite://")

In [9]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    model_format="sentence_transformers",
)
# Important:
# Now that we initialized the Retriever, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation.
# While this can be a time consuming operation (depending on the corpus size), it only needs to be done once.
# At query time, we only need to embed the query and compare it to the existing document embeddings, which is very fast.
document_store.update_embeddings(retriever)


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1


In [10]:

# Write documents to document store
document_store.write_documents(docs)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)


Writing Documents: 10000it [00:00, 54486.27it/s]         
INFO:haystack.document_stores.faiss:Updating embeddings for 49 docs...
Updating Embedding:   0%|          | 0/49 [00:00<?, ? docs/s]INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:hayst

In [11]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [12]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)


In [28]:
prediction = pipe.run(
    query="what is pod",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.02it/s]
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_ha

In [29]:
from pprint import pprint

print(prediction)


{'query': 'what is pod', 'no_ans_gap': 6.876414775848389, 'answers': [<Answer {'answer': 'independent', 'type': 'extractive', 'score': 0.7736260294914246, 'context': 'earch for POD – click on add at lookup values – add POD\nname. (POD is independent- doesn’t rely on any other values) – save.\n➢ Export CSV – Export CSV', 'offsets_in_document': [{'start': 250, 'end': 261}], 'offsets_in_context': [{'start': 70, 'end': 81}], 'document_id': '79046f7fe6c4675b2c84937edcb9a2dc', 'meta': {'name': 'Revised ConvertRite User Manual - New_V3.pdf', '_split_id': 2, 'vector_id': '21'}}>, <Answer {'answer': 'Product Oriented Delivery', 'type': 'extractive', 'score': 0.19494059681892395, 'context': 'ject code) are\nMaster data.\nSTEP 1:\nCreation of POD:\nPOD - The Product Oriented Delivery (POD) model is a software development strategy that centers o', 'offsets_in_document': [{'start': 392, 'end': 417}], 'offsets_in_context': [{'start': 63, 'end': 88}], 'document_id': '8d6b2472231e05f21e8128d114eef203'

In [30]:
from haystack.utils import print_answers

temp = print_answers(prediction,details="minimum" )## Choose from `minimum`, `medium`, and `all`
print(temp)


Query: what is pod
Answers:
[   {   'answer': 'independent',
        'context': 'earch for POD – click on add at lookup values – add POD\n'
                   'name. (POD is independent- doesn’t rely on any other '
                   'values) – save.\n'
                   '➢ Export CSV – Export CSV'},
    {   'answer': 'Product Oriented Delivery',
        'context': 'ject code) are\n'
                   'Master data.\n'
                   'STEP 1:\n'
                   'Creation of POD:\n'
                   'POD - The Product Oriented Delivery (POD) model is a '
                   'software development strategy that centers o'},
    {   'answer': 'Project',
        'context': 'left- admin – role object.\x0c'
                   '- User Manual | Confidential Page | 8\n'
                   '• Select the role name as admin, required POD, Project, '
                   'parent object – Enable flag. '},
    {   'answer': 'Project is related to POD – Hence select the required POD – '
       

In [31]:
for i in temp:
        for key, value in i.items():
            value = value.encode("utf-8")
            print(f"The value of {key} in dictionary {i} is of type {type(value)}")
print(temp)

The value of answer in dictionary {'answer': 'independent', 'context': 'earch for POD – click on add at lookup values – add POD\nname. (POD is independent- doesn’t rely on any other values) – save.\n➢ Export CSV – Export CSV'} is of type <class 'bytes'>
The value of context in dictionary {'answer': 'independent', 'context': 'earch for POD – click on add at lookup values – add POD\nname. (POD is independent- doesn’t rely on any other values) – save.\n➢ Export CSV – Export CSV'} is of type <class 'bytes'>
The value of answer in dictionary {'answer': 'Product Oriented Delivery', 'context': 'ject code) are\nMaster data.\nSTEP 1:\nCreation of POD:\nPOD - The Product Oriented Delivery (POD) model is a software development strategy that centers o'} is of type <class 'bytes'>
The value of context in dictionary {'answer': 'Product Oriented Delivery', 'context': 'ject code) are\nMaster data.\nSTEP 1:\nCreation of POD:\nPOD - The Product Oriented Delivery (POD) model is a software development str

In [22]:
# Make sure you have a GPU running
!nvidia-smi


Tue Mar  7 12:54:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.25       Driver Version: 522.25       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   63C    P8    N/A /  N/A |      0MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------